# Reseau de neuronne

## Importation des librairies

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report

## I. Lecture des descripteurs

In [4]:
data = pd.read_csv('../../predict/model/output.csv')
data = np.array(data.values)
label_names = data[:,1024]
unique_label = np.unique(label_names)
feature_values = data[:, :1024]

## II. Réseau de neuronne

On commence par séparer les données en deux ensembles: un pour l'apprentissage et l'autre pour l'évaluation du modèle

In [5]:
# split données
X_train, X_test, y_train, y_test = train_test_split(feature_values, label_names, test_size=0.33, random_state=43)

On poursuit en transformant les labels en indice:

In [6]:
# enc labelNames to indices
encName2Ind = preprocessing.LabelEncoder()
encName2Ind.fit(unique_label)
labelIndices_unique = encName2Ind.transform(unique_label)
labelIndices  = encName2Ind.transform(label_names)

# Conversion des noms des labels en indices
labelInd_train = encName2Ind.transform(y_train)
labelInd_test = encName2Ind.transform(y_test)

X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(labelInd_train, dtype=tf.float32)
y_test = tf.convert_to_tensor(labelInd_test, dtype=tf.float32)

2023-01-05 20:30:07.868450: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-05 20:30:07.868491: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-05 20:30:07.868530: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nicolas-Lenovo-IdeaPad-C340-14IWL): /proc/driver/nvidia/version does not exist
2023-01-05 20:30:07.868989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


On normalise les données pour réduire la complexité et donner le même poids à tous les descripteurs.

In [7]:
# Normalisation 
images_mean = np.mean(X_train, axis=0, keepdims=True)
images_std = np.std(X_test, axis=0, keepdims=True) + 1e-7
X_train = (X_train - images_mean) / images_std
X_test = (X_test - images_mean) / images_std

/home/nicolas/Documents/3A_ENSTA/ml_embedded/vultr-tensorflow/lib/python3.8/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


On crée ensuite le réseau de neuronne.

In [9]:
## creation du reseau de neuronne
model = Sequential(name="Sequential_CNN")
# couche de classification
model.add(Dense(256, activation='relu', input_shape = [1024]))
model.add(Dense(len(unique_label), activation='softmax'))
model.summary()

Model: "Sequential_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 256)               262400    
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                                 
Total params: 264,970
Trainable params: 264,970
Non-trainable params: 0
_________________________________________________________________


Apprentissage :

In [10]:
# paramètres
NB_EPOCHS = 40
LEARNING_RATE = 0.1
MOMENTUM = 0.09
DECAY = 0.01

#  Choix de la tâche à effectuer = choix de la fonction de loss/pertes/coûts à minimise
loss_fcn = tf.keras.losses.SparseCategoricalCrossentropy()# sans alias

# compilation ...
model.compile(loss=loss_fcn, optimizer='ADAM', metrics=['accuracy'])
#model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_asbolute_error', 'mean_squared_error'])

#... et entrainement
history = model.fit(X_train, y_train, epochs=NB_EPOCHS)

Epoch 1/40
21/21 [==============================] - 1s 7ms/step - loss: 2.0440 - accuracy: 0.2720
Epoch 2/40
21/21 [==============================] - 0s 7ms/step - loss: 1.5550 - accuracy: 0.4664
Epoch 3/40
21/21 [==============================] - 0s 7ms/step - loss: 1.3841 - accuracy: 0.5351
Epoch 4/40
21/21 [==============================] - 0s 7ms/step - loss: 1.2455 - accuracy: 0.5770
Epoch 5/40
21/21 [==============================] - 0s 8ms/step - loss: 1.1351 - accuracy: 0.6293
Epoch 6/40
21/21 [==============================] - 0s 8ms/step - loss: 1.0776 - accuracy: 0.6323
Epoch 7/40
21/21 [==============================] - 0s 8ms/step - loss: 1.0037 - accuracy: 0.6682
Epoch 8/40
21/21 [==============================] - 0s 7ms/step - loss: 0.9275 - accuracy: 0.7160
Epoch 9/40
21/21 [==============================] - 0s 7ms/step - loss: 0.8652 - accuracy: 0.7519
Epoch 10/40
21/21 [==============================] - 0s 8ms/step - loss: 0.8540 - accuracy: 0.7280
Epoch 11/40
21/21 [

## III. Evaluation du modèle

In [12]:
#Confution Matrix and Classification Report
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
print()
print()
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print()
print()
print('Classification Report')
print(classification_report(labelInd_test, y_pred, target_names=unique_label))

11/11 [==============================] - 0s 3ms/step


Confusion Matrix
[[13  0  3  2  1  0  1  0  1  9]
 [ 0 28  1  0  0  0  1  0  0  0]
 [ 2  1 19  4  3  0  0  1  1  6]
 [ 1  1  2 20  2  0  7  3  0  6]
 [ 0  1  2  2 24  0  0  3  1  3]
 [ 0 13  3  1  0  8  3  0  0  2]
 [ 1  0  0  1  1  0 26  0  0  3]
 [ 0  2  0  2  0  0  0 28  0  0]
 [ 1  1  6  3 12  0  3  1  6  1]
 [ 2  2  2  3  0  1  6  0  0 11]]


Classification Report
              precision    recall  f1-score   support

       blues       0.65      0.43      0.52        30
   classical       0.57      0.93      0.71        30
     country       0.50      0.51      0.51        37
       disco       0.53      0.48      0.50        42
      hiphop       0.56      0.67      0.61        36
        jazz       0.89      0.27      0.41        30
       metal       0.55      0.81      0.66        32
         pop       0.78      0.88      0.82        32
      reggae       0.67      0.18      0.28        34
        rock       0.27      0.4

## IV. Enregistrement du modèle sous tensorflow-lite

In [13]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('../../predict/model/model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpxwacl_cp/assets


INFO:tensorflow:Assets written to: /tmp/tmpxwacl_cp/assets
2023-01-05 20:34:58.480979: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-05 20:34:58.481083: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-05 20:34:58.482560: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpxwacl_cp
2023-01-05 20:34:58.484706: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-05 20:34:58.484809: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpxwacl_cp
2023-01-05 20:34:58.491478: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-01-05 20:34:58.493049: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-01-05 20:34:58.570828: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio